In [18]:
import pandas as pd
df = pd.read_csv("All Months-Days Worked Data.csv")
file_path = "../Childrens Results/Results By Year/20230824- Children Database  - English Main.xlsx"
sheet_name = "Children Results "
children = pd.read_excel(file_path, sheet_name=sheet_name)
youth = pd.read_csv("20230904 - Combined Youth Data.csv")
youth = pd.read_csv("2023 Youth Hours & Days-Days Worked Data.csv")
youth = pd.read_csv("2023 Youth Stipends-Stipends Main Data.csv")
file_path = "../Childrens Results/Results By Year/20230824- Children Database  - English Main.xlsx"
sheet_name = "Children Results "
children = pd.read_excel("20230905 - Online Session Tracker w IDs.xlsx, sheet_name=sheet_name)


/Users/jimmckeown/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [7]:
children.columns

Index(['Mcode', 'General Notes', 'School', 'City', 'Centre Type', 'Class',
       'Teacher', 'Grade', 'Hub', 'Classlist Status', 'Language', 'Assessed',
       'Surname', 'Name', 'Full Name', 'Gender', 'LC Name', 'On The Programme',
       'Pair', 'Start Date', 'Total Sessions', 'Start Date Notes',
       'Levels Notes', 'Sessions Notes', 'Assessments Notes',
       'Beginning Level', 'Latest Level', 'Jan - Listen First Sound',
       'Jan - Listen Word', 'Jan - Phonics', 'Jan - Blends',
       'Jan - Sight Words', 'Jan - Read Sentences', 'Jan - Read Story',
       'Jan - Story Comprehension', 'Jan - Written Letters',
       'Jan - Write Name', 'Jan - Write CVCs', 'Jan - Write Sentences',
       'Jan - Total', 'Captured By', 'June - Listen First Sound',
       'June - Listen Word', 'June - Phonics', 'June - Blends',
       'June - Sight Words', 'June - Read Sentences', 'June - Read Story',
       'June - Story Comprehension', 'June - Written Letters',
       'June - Write Name', 'June 

In [5]:
df['Total Days Worked'] = df['March - Days'] + df['April - Days'] + df['May - Days'] + df['June - Days'] + df['July - Days'] + df['August - Days']

In [9]:
sessions_df = children.groupby('LC Name')['Total Sessions'].sum().reset_index()
sessions_df.set_index('LC Name', inplace=True)


In [10]:
df.columns

Index(['Employee ID', 'Combined Link', 'Stipends Link', 'First Names',
       'Last Name', 'Employment Status', 'Year of Service', 'Job Title',
       'Aug Pay Rate', 'Start Date', 'March - Days', 'March - Hours',
       'March - Stipend', 'March Stipend Amount', 'March Amount Diff',
       'April - Days', 'April - Hours', 'April - Stipend',
       'April Stipend Amount', 'April Amount Diff', 'May - Days',
       'May - Hours', 'May - Stipend', 'May Stipend Amount', 'June - Days',
       'June - Hours', 'June - Stipend', 'June Stipend Amount', 'July - Days',
       'July - Hours', 'July - Stipend', 'July Stipend Amount',
       'August - Days', 'August - Hours', 'End Date'],
      dtype='object')

# Looking at some turnover stats

### Youth that started in 2023

In [19]:
import pandas as pd


# Convert Start Date and End Date columns to datetime format
youth['Start Date'] = pd.to_datetime(youth['Start Date'])
youth['End Date'].fillna(pd.Timestamp.today(), inplace=True)  # Fill missing End Dates with today's date
youth['End Date'] = pd.to_datetime(youth['End Date'])

# Calculate number of months active for each youth using years and months
youth['Months Active'] = (youth['End Date'].dt.year - youth['Start Date'].dt.year) * 12 + \
                        (youth['End Date'].dt.month - youth['Start Date'].dt.month)

# Filter youths that started in 2023 and exclude those with 'Did Not Start' in the 'Reason for Leaving' column
youth_2023 = youth[youth['Start Date'].dt.year == 2023]
youth_2023_excluded = youth_2023[youth_2023['Reason for Leaving'] != 'Did Not Start']

# Calculate turnover rate for youths that started in 2023 excluding 'Did Not Start'
total_youth_2023_excluded = len(youth_2023_excluded)
youth_2023_excluded_left = len(youth_2023_excluded[youth_2023_excluded['End Date'].dt.date != pd.Timestamp.today().date()])
turnover_rate_2023_excluded = (youth_2023_excluded_left / total_youth_2023_excluded) * 100

print(f"Turnover rate for youths who started in 2023: {turnover_rate_2023_excluded:.2f}%")

# Breakdown by Job Title
grouped_data = youth_2023_excluded.groupby('Job Title').apply(lambda x: pd.Series({
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date != pd.Timestamp.today().date()]) / len(x)) * 100
})).reset_index()

print(grouped_data)


Turnover rate for youths who started in 2023: 22.58%
            Job Title  Average Months Active  Turnover Rate
0  1000 Stories Youth               2.869565      39.130435
1       EduTech Coach               4.166667      33.333333
2      Literacy Coach               4.620438      23.357664
3  Sport & Arts Coach               4.500000      25.000000
4            Yeboneer               5.183486      17.431193


### All Youth That Worked in 2023

In [41]:
# Filter youths that either ended their employment in 2023 or are still working (no 'End Date')
youth_2023_pool = youth[(youth['End Date'].dt.year == 2023) | (youth['End Date'].dt.date == pd.Timestamp.today().date())]

# Exclude those with 'Did Not Start' in the 'Reason for Leaving' column
youth_2023_pool_excluded = youth_2023_pool[youth_2023_pool['Reason for Leaving'] != 'Did Not Start']

# Calculate turnover rate for the pool of youths for 2023 excluding 'Did Not Start'
total_youth_2023_pool_excluded = len(youth_2023_pool_excluded)
youth_2023_pool_excluded_left = len(youth_2023_pool_excluded[youth_2023_pool_excluded['End Date'].dt.date != pd.Timestamp.today().date()])
turnover_rate_2023_pool_excluded = (youth_2023_pool_excluded_left / total_youth_2023_pool_excluded) * 100

# Amend the grouped_data calculation to include a count of youth for each 'Job Title'
grouped_data_2023_pool_with_count = youth_2023_pool_excluded.groupby('Job Title').apply(lambda x: pd.Series({
    'Number of Youth': len(x),
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date != pd.Timestamp.today().date()]) / len(x)) * 100
})).reset_index()

print(f"Turnover rate for youths who worked in 2023: {turnover_rate_2023_excluded:.2f}%")


# Round the 'Average Months Active' and 'Turnover Rate' to one decimal
grouped_data_2023_pool_with_count['Average Months Active'] = grouped_data_2023_pool_with_count['Average Months Active'].round(1)
grouped_data_2023_pool_with_count['Turnover Rate'] = grouped_data_2023_pool_with_count['Turnover Rate'].round(1)

grouped_data_2023_pool_with_count


Turnover rate for youths who worked in 2023: 22.58%


,Job Title,Number of Youth,Average Months Active,Turnover Rate
0,1000 Stories Youth,37.0,2.9,24.3
1,EduTech Coach,10.0,13.1,30.0
2,Librarian,13.0,21.6,53.8
3,Literacy Coach,209.0,9.5,23.4
4,Sport & Arts Coach,6.0,5.6,16.7
5,Yeboneer,113.0,5.2,16.8


### 2022 Turnover Numbers relative to today.

In [39]:
# Filter youths that started between January and October of 2022
youth_2022 = youth[(youth['Start Date'].dt.year == 2022)]

# Calculate turnover rate for youths that started between January and October of 2022
total_youth_2022 = len(youth_2022)
youth_2022_left = len(youth_2022[youth_2022['End Date'].dt.date != pd.Timestamp.today().date()])
turnover_rate_2022 = (youth_2022_left / total_youth_2022) * 100

# Update the grouped_data calculation for 2022 data
grouped_data_2022 = youth_2022.groupby('Job Title').apply(lambda x: pd.Series({
    'Number of Youth': len(x),
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date != pd.Timestamp.today().date()]) / len(x)) * 100
})).reset_index()

# Round the 'Average Months Active' and 'Turnover Rate' to one decimal
grouped_data_2022['Average Months Active'] = grouped_data_2022['Average Months Active'].round(1)
grouped_data_2022['Turnover Rate'] = grouped_data_2022['Turnover Rate'].round(1)

print(grouped_data_2022)

            Job Title  Number of Youth  Average Months Active  Turnover Rate
0       EduTech Coach              4.0                    8.0           75.0
1           Librarian              6.0                   16.2           33.3
2      Literacy Coach            105.0                   10.5           64.8
3  Sport & Arts Coach              1.0                   10.0            0.0
4            Yeboneer            107.0                    5.1          100.0


In [32]:
# Filter youths that started between January and October of 2022
youth_2022 = youth[(youth['Start Date'].dt.year == 2022) & (youth['Start Date'].dt.month <= 10)]

# Calculate turnover rate for youths that started between January and October of 2022
total_youth_2022 = len(youth_2022)
youth_2022_left = len(youth_2022[youth_2022['End Date'].dt.date < pd.to_datetime("2022/11/15").date()])
turnover_rate_2022 = (youth_2022_left / total_youth_2022) * 100

# Update the grouped_data calculation for 2022 data
grouped_data_2022_nov = youth_2022.groupby('Job Title').apply(lambda x: pd.Series({
    'Number of Youth': len(x),
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date < pd.to_datetime("2022/11/15").date()]) / len(x)) * 100
})).reset_index()

grouped_data_2022_nov



,Job Title,Number of Youth,Average Months Active,Turnover Rate
0,EduTech Coach,4.0,8.000000,50.000000
1,Librarian,6.0,16.166667,0.000000
2,Literacy Coach,105.0,10.466667,23.809524
3,Yeboneer,107.0,5.121495,30.841121


### Including 2020, 2021, and 2022

In [40]:
# Filter youths that worked in 2022 (note this logic only works b/c Tumelo's database starts w/ youth that worked in 2022
youth_2020_to_2022 = youth[youth['Start Date'].dt.year.isin([2020, 2021, 2022]) & 
                          (youth['Start Date'].dt.month <= 12) | 
                          (youth['Start Date'].dt.year < 2022)]

# Calculate turnover rate for youths that started between January 2020 and October 2022
total_youth_2020_to_2022 = len(youth_2020_to_2022)
youth_2020_to_2022_left = len(youth_2020_to_2022[youth_2020_to_2022['End Date'].dt.date < pd.to_datetime("2022/11/15").date()])
turnover_rate_2020_to_2022 = (youth_2020_to_2022_left / total_youth_2020_to_2022) * 100

# Update the grouped_data calculation for 2020 to 2022 data
grouped_data_2020_to_2022_nov = youth_2020_to_2022.groupby('Job Title').apply(lambda x: pd.Series({
    'Number of Youth': len(x),
    'Average Months Active': x['Months Active'].mean(),
    'Turnover Rate': (len(x[x['End Date'].dt.date < pd.to_datetime("2022/11/15").date()]) / len(x)) * 100
})).reset_index()

# Round the 'Average Months Active' and 'Turnover Rate' to one decimal
grouped_data_2020_to_2022_nov['Average Months Active'] = grouped_data_2020_to_2022_nov['Average Months Active'].round(1)
grouped_data_2020_to_2022_nov['Turnover Rate'] = grouped_data_2020_to_2022_nov['Turnover Rate'].round(1)

print(turnover_rate_2020_to_2022)
grouped_data_2020_to_2022_nov


28.000000000000004


,Job Title,Number of Youth,Average Months Active,Turnover Rate
0,EduTech Coach,7.0,17.9,28.6
1,Librarian,19.0,23.5,5.3
2,Literacy Coach,141.0,14.9,29.1
3,Sport & Arts Coach,1.0,10.0,0.0
4,Yeboneer,107.0,5.1,30.8


### Figuring out the % of youth that have worked 6 months or longer.

In [27]:
# Calculate the cutoff date for 6 months earlier than today
cutoff_date = pd.Timestamp.today() - pd.Timedelta(days=6*30)  # Using an approximate value of 30 days per month

# Filter out youth who started after the cutoff date
youth_before_cutoff = youth[youth['Start Date'] < cutoff_date]

# Calculate the number of youth who have worked 6 months or longer
youth_worked_6_months_or_longer = len(youth_before_cutoff[youth_before_cutoff['Months Active'] >= 6])

# Calculate the percentage of all youth that have worked 6 months or longer
percentage_worked_6_months_or_longer = (youth_worked_6_months_or_longer / len(youth_before_cutoff)) * 100

percentage_worked_6_months_or_longer


70.7943925233645